In [10]:
%cd /home/thucth/thucth/project/id_occlusion

/home/thucth/thucth/project/id_occlusion


In [11]:
import argparse
import logging
import os
import time
from collections import OrderedDict
from contextlib import suppress
from datetime import datetime
from tty import CFLAG
import torch
import torch.nn as nn
import torchvision.utils
import yaml
from torch.nn.parallel import DistributedDataParallel as NativeDDP
import matplotlib.pyplot as plt

from src import utils
from src.models.models import create_model, safe_model_name, resume_checkpoint, model_parameters
from src.metrics.losses import *
from src.optim import create_optimizer_v2, optimizer_kwargs
from src.scheduler import create_scheduler

torch.cuda.set_device(1)
_logger = logging.getLogger('train')


## load model

In [13]:
# create model
model = create_model(
    'efficientnet_b0',
    num_classes=1,
    in_chans=3,
    pretrained=False,
    checkpoint_path="/home/thucth/thucth/project/id_occlusion/outputs/efficientnet_b0/v6_more_out_in_224/model_best.pth.tar")
model.cuda()

model.eval()

EfficientNet(
  (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNormAct2d(
    32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
    (drop): Identity()
    (act): SiLU(inplace=True)
  )
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNormAct2d(
          32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
          (drop): Identity()
          (act): SiLU(inplace=True)
        )
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
          (gate): Sigmoid()
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNormAct2d(
      

## Calibration

In [14]:
from src import utils
from src.models.models import create_model, safe_model_name, resume_checkpoint, model_parameters
from src.datasets.load_data import *
from src.datasets import preprocess
from src.metrics.losses import *
from src.optim import create_optimizer_v2, optimizer_kwargs
from src.scheduler import create_scheduler
from src.utils import get_parser

from src.datasets.load_data import *
from src.datasets import preprocess
# create the train and eval datasets
input_size = (224, 224, 3)
mean = None
std = None
val_ds_path = "./resources/data_occlusion_val.csv"
validation_batch_size = 64
workers = 16

tg_tf = preprocess.create_target_tf()
val_tf = preprocess.create_val_tf(input_size=input_size[:-1], means=mean, stds=std)


val_ds = DatasetFromCsv(annotations_file=val_ds_path,
                        transform=val_tf,
                        target_transform=tg_tf)

val_loader = create_dataloader(val_ds,
                               batch_size=validation_batch_size,
                               workers=workers,
                               shuffle=True)


In [15]:
from tqdm import tqdm
labels=[]
outputs=[]

for img, label in tqdm(val_loader):
    labels.extend(label.squeeze().tolist())
    img, label = img.cuda(), label.cuda()
    output=model(img)
    output = output.detach().cpu().numpy().squeeze().tolist()
    outputs.extend(output)

100%|██████████| 39/39 [00:06<00:00,  5.86it/s]


In [17]:
import numpy as np
outputs = np.array(outputs)
labels = np.array(labels)

for threshold in range(0,100, 5):
    output_neg = outputs < threshold
    output_pos = outputs >= threshold
    label_neg = labels < threshold
    label_pos = labels >= threshold

    precision = np.sum(output_pos & label_pos) / np.sum(output_pos)
    recall = np.sum(output_pos & label_pos) / np.sum(label_pos)
    print("="*20)
    print("threshold: %s - precision: %s - recall: %s" % (threshold, precision, recall))


threshold: 0 - precision: 1.0 - recall: 0.984947111472742
threshold: 5 - precision: 0.9572763684913218 - recall: 0.89625
threshold: 10 - precision: 0.9511331444759207 - recall: 0.9592857142857143
threshold: 15 - precision: 0.9664478482859227 - recall: 0.9464285714285714
threshold: 20 - precision: 0.9728506787330317 - recall: 0.9214285714285714
threshold: 25 - precision: 0.9160125588697017 - recall: 0.9725
threshold: 30 - precision: 0.9376518218623482 - recall: 0.965
threshold: 35 - precision: 0.9494195688225538 - recall: 0.9541666666666667
threshold: 40 - precision: 0.9638242894056848 - recall: 0.9325
threshold: 45 - precision: 0.8478847884788479 - recall: 0.942
threshold: 50 - precision: 0.8791946308724832 - recall: 0.917
threshold: 55 - precision: 0.9131334022750776 - recall: 0.883
threshold: 60 - precision: 0.9418472063854048 - recall: 0.826
threshold: 65 - precision: 0.666232073011734 - recall: 0.8516666666666667
threshold: 70 - precision: 0.7122861586314152 - recall: 0.76333333333

/tmp/ipykernel_74098/3069430698.py:11: RuntimeWarning: invalid value encountered in long_scalars
  precision = np.sum(output_pos & label_pos) / np.sum(output_pos)
/tmp/ipykernel_74098/3069430698.py:12: RuntimeWarning: invalid value encountered in long_scalars
  recall = np.sum(output_pos & label_pos) / np.sum(label_pos)
